In [45]:
import pandas as pd
import numpy as np
import plotly.express as px
import folium 
import requests

In [205]:
df_aviones = pd.read_pickle('C:/Users/34670/Desktop/python/Hack a boss/proyecto_3/streamlit/aviones_limpio.pkl')


In [206]:
print(df_aviones.columns)

Index(['codigo_aerolinea', 'fecha', 'numero_vuelo', 'numero_cola',
       'hora_salida_programada', 'hora_salida_real',
       'duracion_programada_vuelo', 'duracion_real', 'retraso_salida',
       'hora_despegue', 'tiempo_pista_salida', 'tiempo_retraso_aerolinea',
       'tiempo_retraso_clima', 'tiempo_retraso_sistema_aviacion',
       'tiempo_retraso_seguridad', 'retraso_llegada_aeronave', 'ciudad_origen',
       'estado_origen', 'aeropuerto_origen', 'dia_semana', 'año',
       'fin_de_semana', 'festivos', 'aeropuerto_destino', 'distancia_millas',
       'ciudad_destino', 'estado_destino', 'retraso_detectado'],
      dtype='object')


In [80]:
print(df_aviones.head())

  codigo_aerolinea      fecha  numero_vuelo numero_cola  \
0        Envoy Air 2021-12-01          3632      N202NN   
1        Envoy Air 2021-12-01          3647      N653AE   
2        Envoy Air 2021-12-01          3723      N616AE   
3        Envoy Air 2021-12-01          4192      N681AE   
4        Envoy Air 2021-12-01          4311      N687JS   

  hora_salida_programada hora_salida_real  duracion_programada_vuelo  \
0                  08:11            08:16                         57   
1                  06:45            06:37                         56   
2                  14:47            14:38                         54   
3                  12:35            12:35                         61   
4                  18:36            19:47                         62   

   duracion_real  retraso_salida hora_despegue  ...  aeropuerto_origen  \
0             69               5         08:33  ...   Abilene Regional   
1             61              -8         06:50  ...   Abilene Re

In [91]:
valores_unicos_aeropuertos = df_aviones['aeropuerto_destino'].unique()
print(valores_unicos_aeropuertos)

print(f"Número de aeropuertos únicos: {len(valores_unicos_aeropuertos)}")


['Dallas/Fort Worth International' "Chicago O'Hare International"
 'Charlotte Douglas International' 'Ronald Reagan Washington National'
 'Philadelphia International' 'Hartsfield-Jackson Atlanta International'
 'William P Hobby' 'Baltimore/Washington International Thurgood Marshall'
 'Orlando International' 'Nashville International'
 'Cincinnati/Northern Kentucky International' 'Indianapolis International'
 'Pittsburgh International' 'Bishop International'
 'Washington Dulles International' 'Norfolk International'
 'Miami International' 'Austin - Bergstrom International'
 'Phoenix Sky Harbor International' 'Charlottesville Albemarle'
 'Shreveport Regional' 'Springfield-Branson National' 'Valdosta Regional'
 'Albert J Ellis' 'Baton Rouge Metropolitan/Ryan Field'
 'Central Il Regional Airport at Bloomington' 'Key West International'
 'Blue Grass' 'West Virginia International Yeager' 'Lovell Field'
 'Southwest Georgia Regional' 'Dothan Regional'
 'Augusta Regional at Bush Field' 'Lehigh V

In [92]:
df_origen = df_aviones[['aeropuerto_origen', 'ciudad_origen', 'estado_origen']].copy()
df_destino = df_aviones[['aeropuerto_destino']].copy()
df_destino.columns = ['aeropuerto_origen']

df_destino['ciudad_origen'] = None
df_destino['estado_origen'] = None
df_aeropuertos_concatenados = pd.concat([df_origen, df_destino])

df_aeropuertos_unicos = df_aeropuertos_concatenados.drop_duplicates(subset=['aeropuerto_origen'])
df_aeropuertos_unicos.reset_index(drop=True, inplace=True)


print(df_aeropuertos_unicos)


                            aeropuerto_origen      ciudad_origen estado_origen
0                            Abilene Regional            Abilene            TX
1                       Akron-Canton Regional              Akron            OH
2    Hartsfield-Jackson Atlanta International            Atlanta            GA
3    Jackson Medgar Wiley Evers International  Jackson/Vicksburg            MS
4                  Jacksonville International       Jacksonville            FL
..                                        ...                ...           ...
358                     Long Island MacArthur              Islip            NY
359             Ithaca Tompkins International    Ithaca/Cortland            NY
360                              Jackson Hole            Jackson            WY
361                         Aberdeen Regional           Aberdeen            SD
362                               Garden City               None          None

[363 rows x 3 columns]


In [2]:
# Garden City que es el único que aparece como none...
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['aeropuerto_origen'] == "Garden City", 'ciudad_origen'] = "Garden City"
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['aeropuerto_origen'] == "Garden City", 'estado_origen'] = "KS"

print(df_aeropuertos_unicos[df_aeropuertos_unicos['aeropuerto_origen'] == "Garden City"])



NameError: name 'df_aeropuertos_unicos' is not defined

In [1]:
df_aeropuertos_unicos.rename(columns={
    'aeropuerto_origen': 'nombre_aeropuerto',
    'ciudad_origen': 'ciudad',
    'estado_origen': 'estado'
}, inplace=True)

print(df_aeropuertos_unicos.columns)


NameError: name 'df_aeropuertos_unicos' is not defined

In [98]:
# API de Foursquare
CLIENT_ID = "QCDWZLNWZBWTQKPLJLC2GCFIGRICGGQX1D1AJD1JUM0FMUPU"
CLIENT_SECRET = "NVRHLER4BNQ2LHGUEWDFPAIF132KBCT2JHQU5X35MLCS1S0B"
API_KEY = "fsq3gHQLqKS5DaSzw9RzKUqjuW9Ec14h06kJE5xVxKSHAfg="

headers = {"Accept": "application/json", "Authorization": API_KEY}

df_aeropuertos_unicos['latitude'] = None
df_aeropuertos_unicos['longitude'] = None
df_aeropuertos_unicos['direccion'] = None

for index, row in df_aeropuertos_unicos.iterrows():
    url_params = {
        "query": "airport" + row['nombre_aeropuerto'],
        "near": f"{row['ciudad']}, {row['estado']}", 'USA'
        "limit": 1
    }

    
    response = requests.get(url="https://api.foursquare.com/v3/places/search", params=url_params, headers=headers)

    
    if response.status_code == 200:
        data = response.json()
        
        if data['results']:
            result = data['results'][0] 
            latitude = result['geocodes']['main']['latitude']
            longitude = result['geocodes']['main']['longitude']
            direccion = result['location']['formatted_address']
           
            df_aeropuertos_unicos.at[index, 'latitude'] = latitude
            df_aeropuertos_unicos.at[index, 'longitude'] = longitude
            df_aeropuertos_unicos.at[index, 'direccion'] = direccion
    else:
        print(f"Error en la fila {index} con el aeropuerto {row['nombre_aeropuerto']}. Respuesta: {response.status_code}")




print(df_aeropuertos_unicos.head())


C:\Users\34670\AppData\Local\Temp\ipykernel_1364\750479686.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aeropuertos_unicos['latitude'] = None
C:\Users\34670\AppData\Local\Temp\ipykernel_1364\750479686.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aeropuertos_unicos['longitude'] = None
C:\Users\34670\AppData\Local\Temp\ipykernel_1364\750479686.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

                          nombre_aeropuerto             ciudad estado  \
0                          Abilene Regional            Abilene     TX   
1                     Akron-Canton Regional              Akron     OH   
2  Hartsfield-Jackson Atlanta International            Atlanta     GA   
3  Jackson Medgar Wiley Evers International  Jackson/Vicksburg     MS   
4                Jacksonville International       Jacksonville     FL   

    latitude  longitude                                          direccion  
0   32.45095 -99.735678        202 Cedar St (North 2nd), Abilene, TX 79601  
1  41.080564 -81.519677                     182 S Main St, Akron, OH 44308  
2  33.640846 -84.432594  6000 N Terminal Pkwy (at Airport Loop Rd), Atl...  
3       None       None                                               None  
4  30.328992 -81.657959  303 N Laura St (Duval St.), Jacksonville, FL 3...  


In [99]:
#Parece que falta alguna coordenada repito proceso a ver si lo encuentra ahora
for index, row in df_aeropuertos_unicos.iterrows():    
    if pd.isnull(row['latitude']) or pd.isnull(row['longitude']) or pd.isnull(row['direccion']):        
        url_params['query'] = "airport " + row['nombre_aeropuerto']

        
        response = requests.get(url="https://api.foursquare.com/v3/places/search", params=url_params, headers=headers)

        
        if response.status_code == 200:
            data = response.json()

            

            if data['results']:
                result = data['results'][0]  
                latitude = result['geocodes']['main']['latitude']
                longitude = result['geocodes']['main']['longitude']
                direccion = result['location']['formatted_address']

                
                df_aeropuertos_unicos.at[index, 'latitude'] = latitude
                df_aeropuertos_unicos.at[index, 'longitude'] = longitude
                df_aeropuertos_unicos.at[index, 'direccion'] = direccion
        else:
            print(f"Error en la segunda búsqueda en la fila {index} con el aeropuerto {row['nombre_aeropuerto']}. Respuesta: {response.status_code}")



print(df_aeropuertos_unicos.head())


                          nombre_aeropuerto             ciudad estado  \
0                          Abilene Regional            Abilene     TX   
1                     Akron-Canton Regional              Akron     OH   
2  Hartsfield-Jackson Atlanta International            Atlanta     GA   
3  Jackson Medgar Wiley Evers International  Jackson/Vicksburg     MS   
4                Jacksonville International       Jacksonville     FL   

    latitude   longitude                                          direccion  
0   32.45095  -99.735678        202 Cedar St (North 2nd), Abilene, TX 79601  
1  41.080564  -81.519677                     182 S Main St, Akron, OH 44308  
2  33.640846  -84.432594  6000 N Terminal Pkwy (at Airport Loop Rd), Atl...  
3  37.974664 -100.871909              1011 N Main St, Garden City, KS 67846  
4  30.328992  -81.657959  303 N Laura St (Duval St.), Jacksonville, FL 3...  


In [100]:
df_aeropuertos_unicos.to_excel('C:/Users/34670/Desktop/python/Hack a boss/proyecto_3/streamlit/proyecto_aviones/df/aeropuertos_unicos.xlsx', index=False)

In [104]:
# Los más famosos creo que sí que están bien, algunos códigos son inventados con iniciales u otras para no repetir que sean únicos como id_unicas
aeropuertos_y_codigos = {
    'Abilene Regional': 'ABR',
    'Akron-Canton Regional': 'AKR',
    'Hartsfield-Jackson Atlanta International': 'ATL',
    'Jackson Medgar Wiley Evers International': 'JMW',
    'Jacksonville International': 'JAX',
    'Albert J Ellis': 'AJE',
    'Jamestown Regional': 'JMR',
    'John Murtha Johnstown-Cambria County': 'JMJ',
    'Joplin Regional': 'JPL',
    'Juneau International': 'JNU',
    'Kahului Airport': 'OGG',
    'Kalamazoo/Battle Creek International': 'AZO',
    'Glacier Park International': 'FCA',
    'Kansas City International': 'MCI',
    'Kearney Regional': 'EAR',
    'Ketchikan International': 'KTN',
    'Key West International': 'EYW',
    'Atlantic City International': 'ACY',
    'Robert Gray AAF': 'GRK',
    'Augusta Regional at Bush Field': 'AGS',
    'King Salmon Airport': 'AKN',
    'McGhee Tyson': 'TYS',
    'Kodiak Airport': 'ADQ',
    'Ellison Onizuka Kona International at Keahole': 'KOA',
    'Ralph Wien Memorial': 'OTZ',
    'La Crosse Regional': 'LSE',
    'Lafayette Regional Paul Fournet Field': 'LFT',
    'Lake Charles Regional': 'LCH',
    'Capital Region International': 'LAN',
    'Laramie Regional': 'LAR',
    'Laredo International': 'LRD',
    'Harry Reid International': 'LAS',
    'Arnold Palmer Regional': 'LBE',
    'Lawton-Fort Sill Regional': 'LAW',
    'Greenbrier Valley': 'LWB',
    'Lewiston Nez Perce County': 'LWS',
    'Austin - Bergstrom International': 'AUS',
    'Blue Grass': 'LEX',
    'Liberal Mid-America Regional': 'LBL',
    'Lihue Airport': 'LIH',
    'Lincoln Airport': 'LNK',
    'Bill and Hillary Clinton Nat Adams Field': 'LIT',
    'Long Beach Airport': 'LGB',
    'East Texas Regional': 'GGG',
    'San Luis Valley Regional/Bergman Field': 'ALS',
    'Los Angeles International': 'LAX',
    'Louisville Muhammad Ali International': 'SDF',
    'Lubbock Preston Smith International': 'LBB',
    'Dane County Regional-Truax Field': 'MSN',
    'Manchester Boston Regional': 'MHT',
    'Manhattan Regional': 'MHK',
    'Marquette Sawyer Regional': 'MQT',
    'Mason City Municipal': 'MCW',
    'Rogue Valley International - Medford': 'MFR',
    'Melbourne International': 'MLB',
    'Memphis International': 'MEM',
    'Key Field': 'MEI',
    'Miami International': 'MIA',
    'Southwest Georgia Regional': 'ABY',
    'Midland International Air and Space Port': 'MAF',
    'General Mitchell International': 'MKE',
    'Minneapolis-St Paul International': 'MSP',
    'Minot International': 'MOT',
    'McAllen International': 'MFE',
    'Missoula International': 'MSO',
    'Canyonlands Field': 'CNY',
    'Mobile Regional': 'MOB',
    'Quad Cities International': 'MLI',
    'Monroe Regional': 'MLU',
    'Monterey Regional': 'MRY',
    'Montgomery Regional': 'MGM',
    'Montrose Regional': 'MTJ',
    'Central Wisconsin': 'CWA',
    'Muskegon County': 'MKG',
    'Myrtle Beach International': 'MYR',
    'Nashville International': 'BNA',
    'Coastal Carolina Regional': 'EWN',
    'Louis Armstrong New Orleans International': 'MSY',
    'John F. Kennedy International': 'JFK',
    'LaGuardia': 'LGA',
    'Newark Liberty International': 'EWR',
    'Meadows Field': 'BFL',
    'New York Stewart International': 'SWF',
    'Newport News/Williamsburg International': 'PHF',
    'Niagara Falls International': 'IAG',
    'Nome Airport': 'OME',
    'Norfolk International': 'ORF',
    'Southwest Oregon Regional': 'OTH',
    'North Platte Regional Airport Lee Bird Field': 'LBF',
    'Metro Oakland International': 'OAK',
    'Ogden-Hinckley': 'OGD',
    'Ogdensburg International': 'OGS',
    'Baltimore/Washington International Thurgood Marshall': 'BWI',
    'Will Rogers World': 'OKC',
    'Eppley Airfield': 'OMA',
    'Albany International': 'ALB',
    'Ontario International': 'ONT',
    'Orlando International': 'MCO',
    'Owensboro Daviess County Regional': 'OWB',
    'Barkley Regional': 'PAH',
    'Pago Pago International': 'PPG',
    'Palm Springs International': 'PSP',
    'Northwest Florida Beaches International': 'ECP',
    'Tri Cities': 'TRI',
    'Pellston Regional Airport of Emmet County': 'PLN',
    'Pensacola International': 'PNS',
    'General Downing - Peoria International': 'PIA',
    'Petersburg James A Johnson': 'PSG',
    'Philadelphia International': 'PHL',
    'Phoenix - Mesa Gateway': 'AZA',
    'Phoenix Sky Harbor International': 'PHX',
    'Pierre Regional': 'PIR',
    'Pittsburgh International': 'PIT',
    'Plattsburgh International': 'PBG',
    'Pocatello Regional': 'PIH',
    'Mercedita': 'PSE',
    'Portland International Jetport': 'PWM',
    'Bangor International': 'BGR',
    'Portland International': 'PDX',
    'Portsmouth International at Pease': 'PSM',
    'Prescott Regional Ernest A Love Field': 'PRC',
    'Rhode Island Tf Green International': 'PVD',
    'Provo Municipal': 'PVU',
    'Pueblo Memorial': 'PUB',
    'Pullman Moscow Regional': 'PUW',
    'Punta Gorda Airport': 'PGD',
    'Raleigh-Durham International': 'RDU',
    'Rapid City Regional': 'RAP',
    'Wiley Post/Will Rogers Memorial': 'BRW',
    'Redding Municipal': 'RDD',
    'Reno/Tahoe International': 'RNO',
    'Baton Rouge Metropolitan/Ryan Field': 'BTR',
    'Rhinelander/Oneida County': 'RHI',
    'Richmond International': 'RIC',
    'Central Wyoming Regional': 'RIW',
    'Roanoke Blacksburg Regional': 'ROA',
    'Rochester International': 'RST',
    'Frederick Douglass Grtr Rochester International': 'ROC',
    'Southwest Wyoming Regional': 'RKS',
    'Chicago/Rockford International': 'RFD',
    'Roswell Air Center': 'ROW',
    'Sacramento International': 'SMF',
    'Jack Brooks Regional': 'BPT',
    'MBS International': 'MBS',
    'Francisco C. Ada Saipan International': 'SPN',
    'Salina Regional': 'SLN',
    'Salt Lake City International': 'SLC',
    'Scott AFB MidAmerica St Louis': 'BLV',
    'San Angelo Regional/Mathis Field': 'SJT',
    'San Antonio International': 'SAT',
    'San Diego International': 'SAN',
    'Bellingham International': 'BLI',
    'San Francisco International': 'SFO',
    'Norman Y. Mineta San Jose International': 'SJC',
    'Luis Munoz Marin International': 'SJU',
    'Bemidji Regional': 'BJI',
    'San Luis County Regional': 'SBP',
    'Orlando Sanford International': 'SFB',
    'John Wayne Airport-Orange County': 'SNA',
    'Roberts Field': 'RDM',
    'Santa Barbara Municipal': 'SBA',
    'Santa Fe Municipal': 'SAF',
    'Santa Maria Public/Capt. G. Allan Hancock Field': 'SMX',
    'Charles M. Schulz - Sonoma County': 'STS',
    'Sarasota/Bradenton International': 'SRQ',
    'Chippewa County International': 'CIU',
    'Savannah/Hilton Head International': 'SAV',
    'Bethel Airport': 'BET',
    'Western Neb. Regional/William B. Heilig Field': 'BFF',
    'Wilkes Barre Scranton International': 'AVP',
    'Billings Logan International': 'BIL',
    'Seattle/Tacoma International': 'SEA',
    'Sheridan County': 'SHR',
    'Shreveport Regional': 'SHV',
    'Sioux Gateway Brig Gen Bud Day Field': 'SUX',
    'Joe Foss Field': 'FSD',
    'Sitka Rocky Gutierrez': 'SIT',
    'South Bend International': 'SBN',
    'Spokane International': 'GEG',
    'Abraham Lincoln Capital': 'SPI',
    'Springfield-Branson National': 'SGF',
    'St. Cloud Regional': 'STC',
    'St George Regional': 'SGU',
    'St Louis Lambert International': 'STL',
    'Greater Binghamton/Edwin A. Link Field': 'BGM',
    'St Pete Clearwater International': 'PIE',
    'University Park': 'SCE',
    'Shenandoah Valley Regional': 'SHD',
    'Stillwater Regional': 'SWO',
    'Stockton Metro': 'SCK',
    'Friedman Memorial': 'SUN',
    'Syracuse Hancock International': 'SYR',
    'Birmingham-Shuttlesworth International': 'BHM',
    'Tallahassee International': 'TLH',
    'Tampa International': 'TPA',
    'Texarkana Regional-Webb Field': 'TXK',
    'Eugene F Kranz Toledo Express': 'TOL',
    'Cherry Capital': 'TVC',
    'Trenton Mercer': 'TTN',
    'Tucson International': 'TUS',
    'Tulsa International': 'TUL',
    'Joslin Field - Magic Valley Regional': 'TWF',
    'Tyler Pounds Regional': 'TYR',
    'Valdosta Regional': 'VLD',
    'Eglin AFB Destin Fort Walton Beach': 'VPS',
    'Vernal Regional': 'VEL',
    'Victoria Regional': 'VCT',
    'Waco Regional': 'ACT',
    'Walla Walla Regional': 'ALW',
    'Ronald Reagan Washington National': 'DCA',
    'Washington Dulles International': 'IAD',
    'Bishop Airport': 'FNT',
    'Waterloo Regional': 'ALO',
    'Watertown Regional': 'ATY',
    'Bismarck Municipal': 'BIS',
    'Pangborn Memorial': 'EAT',
    'Palm Beach International': 'PBI',
    'Westchester County': 'HPN',
    'Sheppard AFB/Wichita Falls Municipal': 'SPS',
    'Wichita Dwight D Eisenhower National': 'ICT',
    'Williston Basin International': 'XWA',
    'New Castle': 'ILG',
    'Wilmington International': 'ILM',
    'Worcester Regional': 'ORH',
    'Wrangell Airport': 'WRG',
    'Yakima Air Terminal/McAllister Field': 'YKM',
    'Yakutat Airport': 'YAK',
    'Yuma MCAS/Yuma International': 'YUM',
    'Central Il Regional Airport at Bloomington': 'BMI',
    'Boise Air Terminal': 'BOI',
    'Logan International': 'BOS',
    'Bozeman Yellowstone International': 'BZN',
    'Brainerd Lakes Regional': 'BRD',
    'Brownsville South Padre Island International': 'BRO',
    'Brunswick Golden Isles': 'BQK',
    'Adak': 'ADK',
    'Albuquerque International Sunport': 'ABQ',
    'Buffalo Niagara International': 'BUF',
    'Bob Hope': 'BUR',
    'Burlington International': 'BTV',
    'Bert Mooney': 'BTM',
    'Cape Girardeau Regional': 'CGI',
    'Casper/Natrona County International': 'CPR',
    'Cedar City Regional': 'CDC',
    'The Eastern Iowa': 'CID',
    'University of Illinois/Willard': 'CMI',
    'Charleston AFB/International': 'CHS',
    'West Virginia International Yeager': 'CRW',
    'Cyril E King': 'STT',
    'Charlotte Douglas International': 'CLT',
    'Rafael Hernandez': 'BQN',
    'Charlottesville Albemarle': 'CHO',
    'Lovell Field': 'CHA',
    'Cheyenne Regional/Jerry Olson Field': 'CYS',
    'Chicago Midway International': 'MDW',
    "Chicago O'Hare International": 'ORD',
    'Alexandria International': 'AEX',
    'Henry E. Rohlsen': 'STX',
    'Cincinnati/Northern Kentucky International': 'CVG',
    'North Central West Virginia': 'CKB',
    'Cleveland-Hopkins International': 'CLE',
    'Yellowstone Regional': 'COD',
    'Easterwood Field': 'CLL',
    'City of Colorado Springs Municipal': 'COS',
    'Columbia Regional': 'COU',
    'Columbia Metropolitan': 'CAE',
    'Lehigh Valley International': 'ABE',
    'Columbus Airport': 'CSG',
    'Golden Triangle Regional': 'GTR',
    'John Glenn Columbus International': 'CMH',
    'Rickenbacker International': 'LCK',
    'Concord Padgett Regional': 'USA',
    'Merle K Mudhole Smith': 'CDV',
    'Corpus Christi International': 'CRP',
    'Dallas Love Field': 'DAL',
    'Dallas/Fort Worth International': 'DFW',
    'James M Cox/Dayton International': 'DAY',
    'Daytona Beach International': 'DAB',
    'Alpena County Regional': 'APN',
    'Deadhorse Airport': 'SCC',
    'Decatur Airport': 'DEC',
    'Del Rio International': 'DRT',
    'Denver International': 'DEN',
    'Rick Husband Amarillo International': 'AMA',
    'Des Moines International': 'DSM',
    'Detroit Metro Wayne County': 'DTW',
    'Devils Lake Regional': 'DVL',
    'Dickinson - Theodore Roosevelt Regional': 'DIK',
    'Dillingham Airport': 'DLG',
    'Dodge City Regional': 'DDC',
    'Dothan Regional': 'DHN',
    'Dubuque Regional': 'DBQ',
    'Duluth International': 'DLH',
    'Durango La Plata County': 'DRO',
    'Eagle County Regional': 'EGE',
    'Chippewa Valley Regional': 'EAU',
    'El Paso International': 'ELP',
    'Ted Stevens Anchorage International': 'ANC',
    'Elko Regional': 'EKO',
    'Elmira/Corning Regional': 'ELM',
    'Erie International/Tom Ridge Field': 'ERI',
    'Delta County': 'ESC',
    'Mahlon Sweet Field': 'EUG',
    'Evansville Regional': 'EVV',
    'Snohomish County': 'PAE',
    'Fairbanks International': 'FAI',
    'Hector International': 'FAR',
    'Northwest Arkansas National': 'XNA',
    'Appleton International': 'ATW',
    'Fayetteville Regional/Grannis Field': 'FAY',
    'Flagstaff Pulliam': 'FLG',
    'Bishop International': 'FNT',
    'Fort Dodge Regional': 'FOD',
    'Fort Lauderdale-Hollywood International': 'FLL',
    'Waynesville-St. Robert Regional Forney Field': 'TBN',
    'Southwest Florida International': 'RSW',
    'Fort Smith Regional': 'FSM',
    'Fort Wayne International': 'FWA',
    'Fresno Yosemite International': 'FAT',
    'Gainesville Regional': 'GNV',
    'California Redwood Coast Humboldt County': 'ACV',
    'Asheville Regional': 'AVL',
    'Gillette Campbell County': 'GCC',
    'Grand Forks International': 'GFK',
    'Central Nebraska Regional': 'GRI',
    'Grand Junction Regional': 'GJT',
    'Gerald R. Ford International': 'GRR',
    'Great Falls International': 'GTF',
    'Green Bay Austin Straubel International': 'GRB',
    'Piedmont Triad International': 'GSO',
    'Pitt Greenville': 'PGV',
    'Greenville-Spartanburg International': 'GSP',
    'Guam International': 'GUM',
    'Gulfport-Biloxi International': 'GPT',
    'Gunnison-Crested Butte Regional': 'GUC',
    'Hagerstown Regional-Richard A. Henson Field': 'HGR',
    'Houghton County Memorial': 'CMX',
    'Valley International': 'HRL',
    'Harrisburg International': 'MDT',
    'Bradley International': 'BDL',
    'Tri-State/Milton J. Ferguson Field': 'HTS',
    'Aspen Pitkin County Sardy Field': 'ASE',
    'Hattiesburg-Laurel Regional': 'PIB',
    'Yampa Valley': 'HDN',
    'Hays Regional': 'HYS',
    'Helena Regional': 'HLN',
    'Range Regional': 'HIB',
    'Hilo International': 'ITO',
    'Hilton Head Airport': 'HHH',
    'Lea County Regional': 'HOB',
    'Daniel K Inouye International': 'HNL',
    'George Bush Intercontinental/Houston': 'IAH',
    'William P Hobby': 'HOU',
    'Huntsville International-Carl T Jones Field': 'HSV',
    'Idaho Falls Regional': 'IDA',
    'Indianapolis International': 'IND',
    'Falls International Einarson Field': 'INL',
    'Ford': 'IMT',
    'Long Island MacArthur': 'ISP',
    'Ithaca Tompkins International': 'ITH',
    'Jackson Hole': 'JAC',
    'Aberdeen Regional': 'ABR',
    'Garden City': 'GCK'
}
df_aeropuertos_unicos['codigo_aeropuerto'] = df_aeropuertos_unicos['nombre_aeropuerto'].map(aeropuertos_y_codigos)



C:\Users\34670\AppData\Local\Temp\ipykernel_1364\434011026.py:366: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aeropuertos_unicos['codigo_aeropuerto'] = df_aeropuertos_unicos['nombre_aeropuerto'].map(aeropuertos_y_codigos)


In [164]:
df_aeropuertos_unicos

,nombre_aeropuerto,ciudad,estado,latitude,longitude,direccion,codigo_aeropuerto
0,Abilene Regional,Abilene,TX,32.45095,-99.735678,"202 Cedar St (North 2nd), Abilene, TX 79601",ABR
1,Akron-Canton Regional,Akron,OH,41.080564,-81.519677,"182 S Main St, Akron, OH 44308",AKR
2,Hartsfield-Jackson Atlanta International,Atlanta,GA,33.640846,-84.432594,"6000 N Terminal Pkwy (at Airport Loop Rd), Atl...",ATL
3,Jackson Medgar Wiley Evers International,Jackson/Vicksburg,MS,37.974664,-100.871909,"1011 N Main St, Garden City, KS 67846",JMW
4,Jacksonville International,Jacksonville,FL,30.328992,-81.657959,"303 N Laura St (Duval St.), Jacksonville, FL 3...",JAX
...,...,...,...,...,...,...,...
358,Long Island MacArthur,Islip,NY,40.729149,-73.212623,"560 Main St, Islip, NY 11751",ISP
359,Ithaca Tompkins International,Ithaca/Cortland,NY,42.600067,-76.180802,"36 Main St, Cortland, NY 13045",ITH
360,Jackson Hole,Jackson,WY,43.47933,-110.763018,"50 W Broadway Ave, Jackson, WY 83001",JAC
361,Aberdeen Regional,Aberdeen,SD,45.463981,-98.487868,"103 1/2 S Main St, Aberdeen, SD 57401",ABR


In [176]:
# Este aeropuerto la coordenada no me cuadraba y la introduzco de forma manual... foorsquare devuelve una cosa pero google me dice que está en otor sitio
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'] == 'Dallas/Fort Worth International', 'latitude'] = 32.897480
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'] == 'Dallas/Fort Worth International', 'longitude'] = -97.040443

direccion_dallas_fort_worth = "2400 Aviation Dr, DFW Airport, TX 75261, USA"  
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'] == 'Dallas/Fort Worth International Airport', 'direccion'] = direccion_dallas_fort_worth



In [196]:
# Este aeropuerto la coordenada no me cuadraba y la introduzco de forma manual... foorsquare devuelve una cosa pero google me dice que está en otor sitio
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'] == 'Ellison Onizuka Kona International at Keahole', 'latitude'] = 19.738889
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'] == 'Ellison Onizuka Kona International at Keahole', 'longitude'] = -156.045556
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'] == 'Ellison Onizuka Kona International at Keahole', 'estado'] = 'HI'

direccion_ellison = "73-200 Kupipi St, Kailua-Kona, HI 96740, USA"  
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'] == 'Ellison Onizuka Kona International at Keahole', 'direccion'] = direccion_ellison

In [197]:
# Este aeropuerto la coordenada no me cuadraba y la introduzco de forma manual... foorsquare devuelve una cosa pero google me dice que está en otor sitio
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'] == 'Guam International', 'latitude'] = 13.48389
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'] == 'Guam International', 'longitude'] = 144.79722
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'] == 'Guam International', 'estado'] = 'GU'

direccion_guam = "355 Chalan Pasaheru B224-A, Tamuning, 96913, Guam"  
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'] == 'Guam International', 'direccion'] = direccion_guam

In [199]:
# Este aeropuerto la coordenada no me cuadraba y la introduzco de forma manual... foorsquare devuelve una cosa pero google me dice que está en otor sitio
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'] == 'Francisco C. Ada Saipan International', 'latitude'] = 15.120255
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'] == 'Francisco C. Ada Saipan International', 'longitude'] = 145.729984
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'] == 'Francisco C. Ada Saipan International', 'estado'] = 'SAIPAN'

direccion_francisco = "PO Box 501055, Saipan, MP 96950-1055​"  
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'] == 'Francisco C. Ada Saipan International', 'direccion'] = direccion_francisco

In [202]:
# Este aeropuerto la coordenada no me cuadraba y la introduzco de forma manual... foorsquare devuelve una cosa pero google me dice que está en otor sitio
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'] == 'Pago Pago International', 'latitude'] = -14.33166
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'] == 'Pago Pago International', 'longitude'] = -170.7115031
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'] == 'Pago Pago International', 'estado'] = 'SAMOA AME'

direccion_samoa = "3 millas al suroeste de Pago Pago, Samoa Americana​"  
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'] == 'Pago Pago International', 'direccion'] = direccion_samoa

In [203]:
# mapa folium usa
mapa = folium.Map(location=[40, -95], zoom_start=4)

# los marcadores, hay que cambiarlos
for index, row in df_aeropuertos_unicos.iterrows():
    folium.Marker(
        [row['latitude'], row['longitude']], 
        popup=f"{row['nombre_aeropuerto']}", 
        tooltip=row['nombre_aeropuerto']
    ).add_to(mapa)

mapa

In [208]:
df_aeropuertos_unicos.to_excel('C:/Users/34670/Desktop/python/Hack a boss/proyecto_3/streamlit/proyecto_aviones/df/aeropuertos_unicos.xlsx', index=False)
df_aviones.to_pickle('C:/Users/34670/Desktop/python/Hack a boss/proyecto_3/streamlit/proyecto_aviones/df/df_aviones_completo.pkl')


In [209]:
# merge para origen
df_aviones = pd.merge(
    df_aviones,
    df_aeropuertos_unicos[['nombre_aeropuerto', 'latitude', 'longitude', 'direccion', 'codigo_aeropuerto']],
    left_on='aeropuerto_origen',
    right_on='nombre_aeropuerto',
    how='left',
    suffixes=(None, '_origen')  # esto es para evita la creación de columnas _x
)



if 'nombre_aeropuerto_origen' in df_aviones.columns:
    df_aviones.drop('nombre_aeropuerto_origen', axis=1, inplace=True)

# merge para destino
df_aviones = pd.merge(
    df_aviones,
    df_aeropuertos_unicos[['nombre_aeropuerto', 'latitude', 'longitude', 'direccion', 'codigo_aeropuerto']],
    left_on='aeropuerto_destino',
    right_on='nombre_aeropuerto',
    how='left',
    suffixes=(None, '_destino')  # esto igual que lo de antes para evitar la creación de columnas _y
)



if 'nombre_aeropuerto_destino' in df_aviones.columns:
    df_aviones.drop('nombre_aeropuerto_destino', axis=1, inplace=True)



In [210]:
print(df_aviones.columns)

Index(['codigo_aerolinea', 'fecha', 'numero_vuelo', 'numero_cola',
       'hora_salida_programada', 'hora_salida_real',
       'duracion_programada_vuelo', 'duracion_real', 'retraso_salida',
       'hora_despegue', 'tiempo_pista_salida', 'tiempo_retraso_aerolinea',
       'tiempo_retraso_clima', 'tiempo_retraso_sistema_aviacion',
       'tiempo_retraso_seguridad', 'retraso_llegada_aeronave', 'ciudad_origen',
       'estado_origen', 'aeropuerto_origen', 'dia_semana', 'año',
       'fin_de_semana', 'festivos', 'aeropuerto_destino', 'distancia_millas',
       'ciudad_destino', 'estado_destino', 'retraso_detectado',
       'nombre_aeropuerto', 'latitude', 'longitude', 'direccion',
       'codigo_aeropuerto', 'latitude_destino', 'longitude_destino',
       'direccion_destino', 'codigo_aeropuerto_destino'],
      dtype='object')


In [211]:
df_aviones.drop('nombre_aeropuerto', axis=1, inplace=True)


# ordenar las columnas para tenerlas mejor visualmente para buscar cositas
nuevo_orden = ['codigo_aerolinea', 'fecha', 'numero_vuelo', 'numero_cola',
               'hora_salida_programada', 'hora_salida_real', 'duracion_programada_vuelo',
               'duracion_real', 'retraso_salida', 'hora_despegue', 'tiempo_pista_salida',
               'tiempo_retraso_aerolinea', 'tiempo_retraso_clima', 'tiempo_retraso_sistema_aviacion',
               'tiempo_retraso_seguridad', 'retraso_llegada_aeronave', 'dia_semana', 'año', 'fin_de_semana', 'festivos',
               'ciudad_origen', 'estado_origen', 'codigo_aeropuerto', 'aeropuerto_origen',
               'latitude', 'longitude', 'direccion', 'ciudad_destino', 'estado_destino', 'codigo_aeropuerto_destino',
               'aeropuerto_destino', 'latitude_destino', 'longitude_destino', 'direccion_destino', 'retraso_detectado', 'distancia_millas']



df_aviones = df_aviones[nuevo_orden]



In [212]:
df_aviones.rename(columns={
    'codigo_aeropuerto': 'codigo_aeropuerto_origen', 
    'latitude': 'latitude_origen',
    'longitude': 'longitude_origen',
    'direccion': 'direccion_origen'
}, inplace=True)



In [213]:
nulos_latitude = df_aviones['latitude_destino'].isnull().sum()
nulos_longitude = df_aviones['longitude_destino'].isnull().sum()

print(f"Valores nulos en 'latitude': {nulos_latitude}")
print(f"Valores nulos en 'longitude': {nulos_longitude}")


Valores nulos en 'latitude': 0
Valores nulos en 'longitude': 0


In [214]:
# Definir la función que implementa la fórmula de Haversine (viene tal cual de chatgpt... parece que funciona)
def haversine(lat1, lon1, lat2, lon2):
    # Convertir latitudes y longitudes de grados a radianes
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    # Diferencia de latitudes y longitudes
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Aplicar la fórmula de Haversine
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    
    # Radio de la Tierra en millas
    R = 3958.8
    distance = R * c
    return distance

# Aplicar la función al DataFrame para calcular la distancia y crear una nueva columna
df_aviones['nueva_distancia_millas'] = df_aviones.apply(lambda row: haversine(row['latitude_origen'], row['longitude_origen'], row['latitude_destino'], row['longitude_destino']), axis=1)


In [190]:
df_aviones.head(5000).to_excel('df_aviones_primeras_500.xlsx', index=False)


In [215]:
print(df_aviones.columns)

Index(['codigo_aerolinea', 'fecha', 'numero_vuelo', 'numero_cola',
       'hora_salida_programada', 'hora_salida_real',
       'duracion_programada_vuelo', 'duracion_real', 'retraso_salida',
       'hora_despegue', 'tiempo_pista_salida', 'tiempo_retraso_aerolinea',
       'tiempo_retraso_clima', 'tiempo_retraso_sistema_aviacion',
       'tiempo_retraso_seguridad', 'retraso_llegada_aeronave', 'dia_semana',
       'año', 'fin_de_semana', 'festivos', 'ciudad_origen', 'estado_origen',
       'codigo_aeropuerto_origen', 'aeropuerto_origen', 'latitude_origen',
       'longitude_origen', 'direccion_origen', 'ciudad_destino',
       'estado_destino', 'codigo_aeropuerto_destino', 'aeropuerto_destino',
       'latitude_destino', 'longitude_destino', 'direccion_destino',
       'retraso_detectado', 'distancia_millas', 'nueva_distancia_millas'],
      dtype='object')


In [216]:
df_aviones.to_pickle('df_aviones_completo.pkl')

In [217]:
df_aviones

,codigo_aerolinea,fecha,numero_vuelo,numero_cola,hora_salida_programada,hora_salida_real,duracion_programada_vuelo,duracion_real,retraso_salida,hora_despegue,...,ciudad_destino,estado_destino,codigo_aeropuerto_destino,aeropuerto_destino,latitude_destino,longitude_destino,direccion_destino,retraso_detectado,distancia_millas,nueva_distancia_millas
0,Envoy Air,2021-12-01,3632,N202NN,08:11,08:16,57,69,5,08:33,...,Dallas/Fort Worth,TX,DFW,Dallas/Fort Worth International,32.89748,-97.040443,"302 Fleming St, Garden City, KS 67846",1,158.0,159.757805
1,Envoy Air,2021-12-01,3647,N653AE,06:45,06:37,56,61,-8,06:50,...,Dallas/Fort Worth,TX,DFW,Dallas/Fort Worth International,32.89748,-97.040443,"302 Fleming St, Garden City, KS 67846",0,158.0,159.757805
2,Envoy Air,2021-12-01,3723,N616AE,14:47,14:38,54,67,-9,14:51,...,Dallas/Fort Worth,TX,DFW,Dallas/Fort Worth International,32.89748,-97.040443,"302 Fleming St, Garden City, KS 67846",0,158.0,159.757805
3,Envoy Air,2021-12-01,4192,N681AE,12:35,12:35,61,59,0,12:44,...,Dallas/Fort Worth,TX,DFW,Dallas/Fort Worth International,32.89748,-97.040443,"302 Fleming St, Garden City, KS 67846",0,158.0,159.757805
4,Envoy Air,2021-12-01,4311,N687JS,18:36,19:47,62,58,71,19:59,...,Dallas/Fort Worth,TX,DFW,Dallas/Fort Worth International,32.89748,-97.040443,"302 Fleming St, Garden City, KS 67846",1,158.0,159.757805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1694237,SkyWest Airlines,2023-12-30,4262,N641CA,16:28,16:51,85,74,23,17:09,...,Minneapolis,MN,MSP,Minneapolis-St Paul International,44.975838,-93.276599,"15 S 9th St (btw Hennepin Ave & Lasalle Ave), ...",0,257.0,255.813380
1694238,SkyWest Airlines,2021-12-31,4244,N931EV,17:31,17:26,83,64,-5,17:35,...,Minneapolis,MN,MSP,Minneapolis-St Paul International,44.975838,-93.276599,"15 S 9th St (btw Hennepin Ave & Lasalle Ave), ...",0,257.0,255.813380
1694239,SkyWest Airlines,2021-12-31,4255,N873AS,05:35,06:33,84,80,58,07:03,...,Minneapolis,MN,MSP,Minneapolis-St Paul International,44.975838,-93.276599,"15 S 9th St (btw Hennepin Ave & Lasalle Ave), ...",1,257.0,255.813380
1694240,SkyWest Airlines,2022-12-31,4299,N922EV,07:00,07:00,78,70,0,07:18,...,Minneapolis,MN,MSP,Minneapolis-St Paul International,44.975838,-93.276599,"15 S 9th St (btw Hennepin Ave & Lasalle Ave), ...",0,257.0,255.813380


In [195]:
# Número de trozos en los que quieres dividir el DataFrame
num_trozos = 5

# Calcular el tamaño de cada trozo
tamaño_trozo = np.ceil(len(df_aviones) / num_trozos).astype(int)

# Dividir y guardar cada trozo
for i in range(num_trozos):
    inicio = i * tamaño_trozo
    fin = inicio + tamaño_trozo
    df_trozo = df_aviones.iloc[inicio:fin]
    df_trozo.to_pickle(f'C:/Users/34670/Desktop/python/Hack a boss/proyecto_3/streamlit/proyecto_aviones/df/pickle/df_aviones_trozo_{i}.pkl')